# Statistical Analysis Project
## Part 5 -  Baysian Inference & Missing Data
#### Adi Hatav and Tamar Dufour Dror

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.stats import beta
import statistics
import random

def r(float):
    return(round(float, 3))

### Baysian Inference

#### In this section, we will analyze the effect of depression (Y) on the respondents BMI (X), among women aged 18-64 (as in part 2).

In [2]:
df = pd.read_csv('CVD_cleaned.csv')
B=400
df["Depression"] = df["Depression"].replace({'No':-1, 'Yes':1})
np.random.seed(101)
ages_catrgories = ['18-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64']
women_df = df[df["Age_Category"].isin(ages_catrgories)]
women_df = women_df[women_df["Sex"]=="Female"]

all_psi = []
all_psi_CI = []

To simulate a situation in which we have past data and new data, we will create two samples from the full data:
1. seen_sample - a sample 200 of observations. We will treat these data as new data.
2. past_sample - a sample of 1000 differnet observation.  We will treat these data as past data.

In [3]:
seen_sample = women_df.sample(n=200)
df_all = women_df.merge(seen_sample, how='left', indicator=True)
not_in_sample = df_all[df_all["_merge"] == "left_only"]
past_sample = not_in_sample.sample(1000).drop("_merge", axis=1)

Let Z be a new binary variable:<br>
    $Z = I_{X>\tau}, \quad \tau = \bar{X_{dep}}+\bar{X_{not_dep}}$

In [4]:
past_sample_dep = past_sample[past_sample["Depression"]==1]["BMI"]
past_sample_non_dep = past_sample[past_sample["Depression"]==-1]["BMI"]
tau = (past_sample_dep.mean()+ past_sample_non_dep.mean())/2
r(tau)

28.981

Define the probability: $P(Z=1|Y=j)=p_j, \quad j=0,1$<br>
We are interested in extimating the log odds ratio $\psi = \eta(p_1)-\eta(p_2)$, when\: $\eta(p)=log\frac{p}{1-p}$.<br>
Therefore: $\psi = log\left(\frac{\frac{p_0}{1-p_0}}{\frac{p_1}{1-p_1}}\right)$

In [5]:
def eta (p):
    return np.log(p/(1-p)) 

p1 = sum([1 if x>= tau else 0 for x in past_sample_dep])/len(past_sample_dep)
p2 = sum([1 if x>= tau else 0 for x in past_sample_non_dep])/len(past_sample_non_dep)
psi = eta(p1)-eta(p2)
all_psi.append(psi)
print("psi =", r(psi))

psis=[]
#Bootstrap
for b in range(B):
    boot_sample = past_sample.sample(n=1000, replace=True)
    boot_dep = boot_sample[boot_sample["Depression"]==1]["BMI"]
    boot_non_dep = boot_sample[boot_sample["Depression"]==-1]["BMI"]
    p1 = sum([1 if x>= tau else 0 for x in boot_dep])/len(boot_dep)
    p2 = sum([1 if x>= tau else 0 for x in boot_non_dep])/len(boot_non_dep)
    psi = eta(p1)-eta(p2)
    psis.append(psi)
psis = np.array(psis)
boots_q0975 = np.quantile(psis, q=0.975)
boots_q0025 = np.quantile(psis, q=0.025)
print("95% credible interval for psi, based on precentile:[", r(boots_q0025), ",", r(boots_q0975),"]")
all_psi_CI.append([r(boots_q0025),r(boots_q0975)]) 
    

psi = 0.624
95% credible interval for psi, based on precentile:[ 0.339 , 0.929 ]


Hence, $\left(\frac{\frac{p_0}{1-p_0}}{\frac{p_1}{1-p_1}}\right)>1$<br>
$\frac{p_0}{1-p_0}>\frac{p_1}{1-p_1}$

Estimating psi according to a uniform prior:<br>
$ f(p|z^n) \propto Beta(S+1, n-S+1)$ <br> $S=\sum Z_i$

In [6]:
n1 = len(past_sample_dep)
n2 = len(past_sample_non_dep)
p1_est = 2/(2+n1)*0.5 + (n1/(n1+2))*p1
p2_est = 2/(2+n2)*0.5 + (n2/(n2+2))*p2
psi_est = eta(p1_est)-eta(p2_est)
psi_est
all_psi.append(psi_est)

In [7]:
s1 = p1*n1
s2 = p2*n2
estimators = []
for b in range(B):
    p1_val = stats.beta.rvs(s1 +1, n1 - s1 + 1, size = 1)
    p2_val = stats.beta.rvs(s2 +1, n2 - s2 + 1, size = 1)
    estimators.append(eta(p1_val)-eta(p2_val))
q0975 = np.quantile(estimators, q=0.975)
q0025 = np.quantile(estimators, q=0.025)
print("95% credible interval for psi, based on uniform prior:[", r(q0025), ", ", r(q0975),"]")
all_psi_CI.append([r(q0025),r(q0975)])

95% credible interval, based on uniform prior:[ 0.452 ,  0.975 ]


Estimating psi according to Jeffreys' prior: <br>
$Z_1,...,Z_n ~ Ber(p),\qquad$ therfore $I(p) = \frac{1}{p(1-p}$<br>
$\pi(p)= \sqrt{\frac{1}{p(1-p}}$<br>
$ f(p_i|z^n) \propto Beta(S_i+0.5, n_i-S_i+0.5)$

In [8]:
p1_lst = []
p2_lst = []
for b in range(B):
    p1_val = stats.beta.rvs(s1 +0.5, n1 - s1 + 0.5, size = 1)
    p2_val = stats.beta.rvs(s2 +0.5, n2 - s2 + 0.5, size = 1)
    p1_lst.append(p1_val)
    p2_lst.append(p2_val)
print("Estimator for p1: ", r(np.array(p1_lst).mean()))
print("Estimator for p2: ", r(np.array(p2_lst).mean()))
psi_C = eta(np.array(p1_lst).mean())-eta(np.array(p2_lst).mean())
print("Estimator for psi: ", r(psi_C))
all_psi.append(psi_C)

Estimator for p1:  0.502
Estimator for p2:  0.328
Estimator for psi:  0.727


In [9]:
estimators1 = []
for b in range(B):
    p1_val = stats.beta.rvs(s1 +0.5, n1 - s1 + 0.5, size = 1)
    p2_val = stats.beta.rvs(s2 +0.5, n2 - s2 + 0.5, size = 1)
    estimators1.append(eta(p1_val)-eta(p2_val))
q0975 = np.quantile(estimators1, q=0.975)
q0025 = np.quantile(estimators1, q=0.025)
print("95% credible interval, based on Jeffreys' prior:[", r(q0025), ",", r(q0975),"]")
all_psi_CI.append([r(q0025), r(q0975)])

95% credible interval, based on Jeffreys' prior:[ 0.42 , 0.988 ]


Calculating prior, based on the past sample:
We know that beta distribution is a conjugate prior for binomial. As a result, the posterior distribution is beta too.<br>
Using the method of moments, we will estimate the parameters of the posterior distribution:<br>
$\mathbb{E}[X] = \frac{\alpha}{\alpha+\beta}$ <br>
$\mathbb{Var}[X] = \frac{\alpha\beta}{(\alpha+\beta)(\alpha+\beta+1)}$ <br>

In [10]:
def estimate_alpha_beta(data):
    mean = data.mean()
    var = data.var()
    alpha = mean*(((mean*(1-mean))/var)-1)
    beta = (1-mean)*((mean*(1-mean)/var)-1)
    return alpha, beta

w_dep = past_sample[past_sample["Depression"]==1]["BMI"].to_numpy()
max_w_dep = np.max(w_dep)
w_dep = w_dep/max_w_dep #normalization
w_non_dep = past_sample[past_sample["Depression"]==-1]["BMI"].to_numpy()
max_w_non_dep = np.max(w_non_dep)
w_non_dep = w_non_dep/max_w_non_dep #normalization


a_dep, b_dep = estimate_alpha_beta(w_dep)
a_non, b_non = estimate_alpha_beta(w_non_dep)

print("The prior of the BMI for women who reported being depressed: Beta(",r(a_dep),",", r(b_dep),")")
print("The prior of BMI of Women who reported not being depressed: Beta(",r(a_non),",", r(b_non),")")

The prior of the BMI for women who reported being depressed: Beta( 8.073 , 16.37 )
The prior of BMI of Women who reported not being depressed: Beta( 10.738 , 15.329 )


In [11]:
estimators2 = []
p1_lst1 = []
p2_lst1 = []
# 
for b in range(B):
    p1_val = stats.beta.rvs(a_dep, b_dep, size = 1)
    p2_val = stats.beta.rvs(a_non, b_non, size = 1)
    p1_lst1.append(p1_val)
    p2_lst1.append(p2_val)
    estimators2.append(eta(p1_val)-eta(p2_val))
q0975 = np.quantile(estimators2, q=0.975)
q0025 = np.quantile(estimators2, q=0.025)
psi_D = eta(np.array(p1_lst1).mean())-eta(np.array(p2_lst1).mean())
all_psi.append(psi_D)

print("Estimator for p1: ", r(np.array(p1_lst1).mean()))
print("Estimator for p2: ", r(np.array(p2_lst1).mean()))
print("psi is: ", r(psi_D))

print("95% credible interval for psi, based on uniform prior:[", r(q0025), ", ", r(q0975),"]")
all_psi_CI.append([r(q0025), r(q0975)])

Estimator for p1:  0.333
Estimator for p2:  0.41
psi is:  -0.329
95% credible interval for psi, based on uniform prior:[ -1.465 ,  0.923 ]


Comparison of the different estimates:

In [12]:
questions = ["A", "B", "C", "D"]
print("Credible intervals and estimator for psi:")
for i, q in enumerate(questions):
    print(q, ":")
    print(r(all_psi[i]))
    print(all_psi_CI[i])

Credible intervals and estimator for psi:
A :
0.624
[0.339, 0.929]
B :
0.716
[0.452, 0.975]
C :
0.727
[0.42, 0.988]
D :
-0.329
[-1.465, 0.923]


### Missing Data
Features: height, smoking history, alcohol consumption <br>
Response variable: weight<br>
In this section, we will artificially generate missing data in the response variable, and examine the success of various methods.

In [13]:
df["Smoking_History"] = df["Smoking_History"].replace({'No':0, 'Yes':1})
data = df[["Height_(cm)","Smoking_History","Alcohol_Consumption", "Weight_(kg)"]]
data = data.sample(n=1000)
X = data[["Height_(cm)","Smoking_History","Alcohol_Consumption"]].to_numpy()
y = data["Weight_(kg)"].to_numpy()
X = np.c_[np.ones(X.shape[0]), X]
beta_index = [0,1,2,3]

Calculating the beta coefficient vector according to the original database:

In [14]:
def calc_beta_coef(X,y):
    beta_CI_dict_2 = {key: None for key in beta_index}
    XTX_inverse = np.linalg.inv(np.matmul(X.T, X))
    XTy = np.matmul(X.T, y)
    n = X.shape[0]
    p=4
    beta_estimate = np.matmul(XTX_inverse,XTy)
    y_mean = np.mean(y)
    y_estimate = np.matmul(X, beta_estimate)

    e = y - y_estimate
    noise_var = 1/(n-p)*sum(i**2 for i in e)
    C = XTX_inverse
    z = stats.norm.ppf(0.975)
    for i in range (beta_estimate.shape[0]):
        se = (noise_var*C[i][i])**0.5
        lower = beta_estimate[i]-(z*se)
        upper = beta_estimate[i]+(z*se)
        print("Estimator fot beta "+str(i),": ", r(beta_estimate[i]))
        print("95% Confidence interval for beta"+ str(i)+ " : [", r(lower), ", ", r(upper),"]")
        beta_CI_dict_2[i] = [lower,upper]
        print("-------------------------------------------------------------------------")
    return beta_estimate
    

calc_beta_coef(X,y)

Estimator fot beta 0 :  -81.948
95% Confidence interval for beta0 : [ -100.646 ,  -63.251 ]
-------------------------------------------------------------------------
Estimator fot beta 1 :  0.965
95% Confidence interval for beta1 : [ 0.855 ,  1.076 ]
-------------------------------------------------------------------------
Estimator fot beta 2 :  2.775
95% Confidence interval for beta2 : [ 0.383 ,  5.166 ]
-------------------------------------------------------------------------
Estimator fot beta 3 :  -0.161
95% Confidence interval for beta3 : [ -0.311 ,  -0.012 ]
-------------------------------------------------------------------------


array([-81.94818344,   0.96539206,   2.774529  ,  -0.16130645])

We will create a database with missing data in which the greater the value of the explanatory variable, the more likely it is to be missing:

In [15]:
data1 = data.sort_values("Weight_(kg)", ascending = False)

y1 = data1["Weight_(kg)"].to_numpy()
y_missing = y1
for i in range(data1.shape[0]):
    ber = stats.bernoulli.rvs(1-(0.9*(i/1000)+0.1))
    if ber ==1:
        y_missing[i] = None

In [16]:
data1["Weight_(kg)"] = y_missing
data_nan = data1.copy()
data1

,Height_(cm),Smoking_History,Alcohol_Consumption,Weight_(kg)
147502,175,1,0,NaN
228942,183,0,8,NaN
29135,193,1,0,176.90
122334,180,1,0,NaN
233629,170,1,0,170.10
...,...,...,...,...
21748,160,0,0,44.00
33433,150,1,0,43.09
166571,155,1,0,40.82
225117,165,1,0,38.56


In [17]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 147502 to 235233
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Height_(cm)          1000 non-null   int64  
 1   Smoking_History      1000 non-null   int64  
 2   Alcohol_Consumption  1000 non-null   int64  
 3   Weight_(kg)          565 non-null    float64
dtypes: float64(1), int64(3)
memory usage: 39.1 KB


Describe the new data, with the missing values:

In [18]:
data1_full = data1.dropna(axis=0)
data1_full.describe()

,Height_(cm),Smoking_History,Alcohol_Consumption,Weight_(kg)
count,565.000000,565.000000,565.000000,565.000000
mean,168.217699,0.375221,4.669027,73.580212
std,10.046190,0.484609,8.062599,16.365648
min,135.000000,0.000000,0.000000,38.560000
25%,160.000000,0.000000,0.000000,61.230000
50%,168.000000,0.000000,1.000000,72.570000
75%,175.000000,1.000000,5.000000,83.910000
max,196.000000,1.000000,30.000000,176.900000


For comparison, describe the original data without missing values:

In [19]:
data.describe()

,Height_(cm),Smoking_History,Alcohol_Consumption,Weight_(kg)
count,1000.00000,1000.000000,1000.000000,1000.000000
mean,170.49700,0.409000,4.799000,83.008940
std,10.82632,0.491895,7.996032,21.555014
min,130.00000,0.000000,0.000000,38.560000
25%,163.00000,0.000000,0.000000,68.040000
50%,170.00000,0.000000,1.000000,81.650000
75%,178.00000,1.000000,5.000000,93.102500
max,241.00000,1.000000,30.000000,204.120000


Calculating the beta coefficient vector according to the full observations in the new database:

In [20]:
X1 = data1_full[["Height_(cm)","Smoking_History","Alcohol_Consumption"]].to_numpy()
X1 = np.c_[np.ones(X1.shape[0]), X1]
y1 = data1_full["Weight_(kg)"].to_numpy()
beta_coef_full_obs = calc_beta_coef(X1,y1)

Estimator fot beta 0 :  -64.732
95% Confidence interval for beta0 : [ -84.545 ,  -44.919 ]
-------------------------------------------------------------------------
Estimator fot beta 1 :  0.821
95% Confidence interval for beta1 : [ 0.703 ,  0.939 ]
-------------------------------------------------------------------------
Estimator fot beta 2 :  1.58
95% Confidence interval for beta2 : [ -0.848 ,  4.008 ]
-------------------------------------------------------------------------
Estimator fot beta 3 :  -0.09
95% Confidence interval for beta3 : [ -0.237 ,  0.058 ]
-------------------------------------------------------------------------


Complete the missing values using regression imputation:<br>

In [21]:
for i in range (data1.shape[0]):
     if pd.isna(data1.iloc[i][3]):
        data1.iloc[i,3] = beta_coef_full_obs[0]+beta_coef_full_obs[1]* data1.iloc[i,0]+beta_coef_full_obs[2]* data1.iloc[i,1]+beta_coef_full_obs[3]* data1.iloc[i,2]
data1

,Height_(cm),Smoking_History,Alcohol_Consumption,Weight_(kg)
147502,175,1,0,80.555096
228942,183,0,8,84.827978
29135,193,1,0,176.900000
122334,180,1,0,84.661019
233629,170,1,0,170.100000
...,...,...,...,...
21748,160,0,0,44.000000
33433,150,1,0,43.090000
166571,155,1,0,40.820000
225117,165,1,0,38.560000


In [22]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 147502 to 235233
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Height_(cm)          1000 non-null   int64  
 1   Smoking_History      1000 non-null   int64  
 2   Alcohol_Consumption  1000 non-null   int64  
 3   Weight_(kg)          1000 non-null   float64
dtypes: float64(1), int64(3)
memory usage: 39.1 KB


Multiple imputation:

In [23]:
data3 = data_nan.copy()
M = 500
sigma_square = data1["Weight_(kg)"].to_numpy().var()
weight_new = np.zeros(data3.shape[0])
betas = np.zeros((M, data3.shape[1]))
varss = np.zeros((M, data3.shape[1]))

def linear_reg(X,y):
    X = np.c_[np.ones(X.shape[0]), X]
    XTX_inverse = np.linalg.inv(np.matmul(X.T, X))
    XTy = np.matmul(X.T, y)
    n = X.shape[0]
    p=4
    beta_estimate = np.matmul(XTX_inverse,XTy)
    return beta_estimate, XTX_inverse
    

for m in range (M):
    for i in range(data3.shape[0]):
        row = data3.iloc[i].to_numpy()
        if pd.isna(row[3]):
            mu = beta_coef_full_obs[0]+beta_coef_full_obs[1]*row[0]+beta_coef_full_obs[2]*row[1]+ beta_coef_full_obs[3]*row[2]
            weight_new[i] += np.random.normal(mu, sigma_square)
        else:
            weight_new[i] += row[3]
    y_curr = (weight_new/(m+1))
    X_curr = data3[["Height_(cm)","Smoking_History","Alcohol_Consumption"]].to_numpy()
    betas[m], C_curr = linear_reg(X_curr,y_curr)
    beta_curr = np.array(betas[m])
    X_curr = np.c_[np.ones(X_curr.shape[0]), X_curr]
    y_estimate_curr = np.matmul(X_curr,beta_curr )
    n = X_curr.shape[0]
    p=4
    e_curr = y_curr - y_estimate_curr
    noise_var_curr = 1/(n-p)*sum(i**2 for i in e_curr)
    
    ses = []
    for j in range (betas[m].shape[0]):
        se_s = (noise_var_curr*C_curr[j][j])
        ses.append(se_s)
    varss[m] = ses
        
weight_new = weight_new/M

data3["Weight_(kg)"] = weight_new
data3

,Height_(cm),Smoking_History,Alcohol_Consumption,Weight_(kg)
147502,175,1,0,62.327339
228942,183,0,8,87.551213
29135,193,1,0,176.900000
122334,180,1,0,76.532607
233629,170,1,0,170.100000
...,...,...,...,...
21748,160,0,0,44.000000
33433,150,1,0,43.090000
166571,155,1,0,40.820000
225117,165,1,0,38.560000


In [24]:
beta_estimate_C, c = linear_reg(data3[["Height_(cm)","Smoking_History","Alcohol_Consumption"]].to_numpy(),weight_new)
beta_estimate_C

array([-5.82200628e+01,  7.81826753e-01,  1.02183191e+00, -4.78001327e-02])

Calculating the estimator to the s.e according to Robin's formula:

In [25]:
beta_estimate_se =np.zeros((beta_estimate_C.shape[0]))

for b in range (beta_estimate_C.shape[0]):
    left1 = 0
    right1 = 0
    for m in range(M):
        left1 += varss[m][b]
        right1 += (betas[m][b]-beta_estimate_C[b])**2
    left1 = (1/M)*left1
    right1 = ((M+1)/(M*(M-1)))*right1
    se = (left1+right1)**0.5
    beta_estimate_se[b] = se
beta_estimate_se

array([11.17592042,  0.0646105 ,  1.48333726,  0.10112337])

In [26]:
beta_CI_dict_D = {key: None for key in beta_index}
z = stats.norm.ppf(0.975)
for i in range (beta_estimate_C.shape[0]):
    lower = beta_estimate_C[i]-(z*beta_estimate_se[i])
    upper = beta_estimate_C[i]+(z*beta_estimate_se[i])
    print("Estimator fot beta", i, r(beta_estimate_C[i]))
    print("Confidence interval 95% for beta"+ str(i)+ " : [",r(lower), ",",r(upper),"]")
    beta_CI_dict_D[i]=[lower, upper]
    print("-----------------------------------------------------------------------------------------")

Estimator fot beta 0 -58.22
Confidence interval 95% for beta0 : [ -80.124 , -36.316 ]
-----------------------------------------------------------------------------------------
Estimator fot beta 1 0.782
Confidence interval 95% for beta1 : [ 0.655 , 0.908 ]
-----------------------------------------------------------------------------------------
Estimator fot beta 2 1.022
Confidence interval 95% for beta2 : [ -1.885 , 3.929 ]
-----------------------------------------------------------------------------------------
Estimator fot beta 3 -0.048
Confidence interval 95% for beta3 : [ -0.246 , 0.15 ]
-----------------------------------------------------------------------------------------


Calculating $P(R=1|X_1,...,X_k)$ where $X_1,...,X_k$ are the explanatory variables, and $R$ is the missing variable (binary):

In [27]:
data4 = data_nan.copy()
data4["Weight_(kg)"] = data4["Weight_(kg)"].apply(lambda x: 1 if not pd.isnull(x) else 0)
data4

,Height_(cm),Smoking_History,Alcohol_Consumption,Weight_(kg)
147502,175,1,0,0
228942,183,0,8,0
29135,193,1,0,1
122334,180,1,0,0
233629,170,1,0,1
...,...,...,...,...
21748,160,0,0,1
33433,150,1,0,1
166571,155,1,0,1
225117,165,1,0,1


In [28]:
import statsmodels.api as stm
X_lr = data4[["Height_(cm)","Smoking_History","Alcohol_Consumption"]].to_numpy()
X_lr = np.c_[np.ones(X_lr.shape[0]), X_lr]
y_lr = data4["Weight_(kg)"]
reg = stm.Logit(y_lr,X_lr).fit()
print(reg.params)
print()
lr_params = reg.params.to_numpy()
pi_W = np.zeros((X_lr.shape[0]))

for i in range (X_lr.shape[0]):
    row = X_lr[i]
    x = lr_params[0]+lr_params[1]*row[1]+lr_params[2]*row[2]+lr_params[3]*row[3]
    numerator = np.exp(x)
    denominator = 1+np.exp(x)
    pi_W[i] = numerator/denominator


Optimization terminated successfully.
         Current function value: 0.652216
         Iterations 5
const    8.624486
x1      -0.048539
x2      -0.284740
x3       0.009313
dtype: float64

